In [ ]:
# install ITSxpress and Q2-itsxpress in the environment
conda install -c bioconda itsxpress
pip install q2-itsxpress

In [ ]:
# refresh plugins
qiime dev refresh-cache

In [ ]:
# check to see if the plugins are installed correctly
qiime itsxpress

In [ ]:
# import ITS2 sequences
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \ 
  --input-path ITS2
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza
# output file: demux-paired-end.qza

In [ ]:
qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv
# output file: demux-paired-end.qzv

In [ ]:
qiime dada2 denoise-paired \
--i-demultiplexed-seqs demux-paired-end.qza \
--p-trim-left-f 0 \
--p-trim-left-r 0 \
--p-trunc-len-f 0 \
--p-trunc-len-r 0 \
--o-representative-sequences BNW-skin-ITS-paired-rep-seqs.qza \
--o-table BNW-skin-ITS-paired-table.qza \
--o-denoising-stats BNW-skin-ITS-paired-stats-dada2.qza

In [ ]:
# create visualisation files for the files above
qiime metadata tabulate \
  --m-input-file BNW-skin-ITS-paired-stats-dada2.qza \
  --o-visualization ITS-BNW-skin-stats-dada2.qzv
qiime metadata tabulate \
  --m-input-file BNW-skin-ITS-paired-table.qza \      
  --o-visualization BNW-skin-ITS-paired-table.qzv
qiime metadata tabulate \
  --m-input-file BNW-skin-ITS-paired-rep-seqs.qza \ 
  --o-visualization BNW-skin-ITS-paired-rep-seqs.qzv

In [ ]:
qiime feature-table group \
  --i-table BNW-skin-ITS-paired-table.qza \
  --p-axis sample \
  --m-metadata-file BNW-skin-metadata-index.tsv \
  --m-metadata-column SampleID-new \
  --p-mode sum \
  --o-grouped-table BNW-skin-ITS-paired-reindexed-table.qza

In [ ]:
qiime feature-table summarize \
  --i-table BNW-skin-ITS-paired-reindexed-table.qza \
  --m-sample-metadata-file BNW-skin-metadata.tsv \
  --o-visualization BNW-skin-ITS-paired-reindexed-table.qzv

# UNITE database

In [ ]:
# download the UNITE QIIME reference sequences and taxonomy files
wget https://files.plutof.ut.ee/public/orig/C5/54/C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz

In [ ]:
# decompress file
tar xzf C5547B97AAA979E45F79DC4C8C4B12113389343D7588716B5AD330F8BDB300C9.tgz

In [ ]:
# move into developer environment
cd sh_qiime_release_10.05.2021/developer/

In [ ]:
# fix formatting errors that prevent importation of the reference sequences. 
# There are white spaces and lower case letters that need to be converted
awk '/^>/ {print($0)}; /^[^>]/ {print(toupper($0))}' sh_refs_qiime_ver8_99_10.05.2021_dev.fasta | tr -d ' ' > sh_refs_qiime_ver8_99_10.05.2021_dev_uppercase.fasta

In [ ]:
# import UNITE reference sequences
qiime tools import \
--type 'FeatureData[Sequence]' \
--input-path sh_refs_qiime_ver8_99_10.05.2021_dev_uppercase.fasta \
--output-path unite-ver8-seqs_99.qza

In [ ]:
# import taxonomy file
qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-path sh_taxonomy_qiime_ver8_99_10.05.2021_dev.txt \
--output-path unite-ver8-taxonomy_99.qza \
--input-format HeaderlessTSVTaxonomyFormat

In [ ]:
# train the classifier
qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads unite-ver8-seqs_99.qza \
--i-reference-taxonomy unite-ver8-taxonomy_99.qza \
--o-classifier unite-ver8-99-classifier.qza

In [ ]:
# taxonomoc classification
qiime feature-classifier classify-sklearn \
  --i-classifier sh_qiime_release_10.05.2021/developer/unite-ver8-99-classifier.qza \
  --i-reads BNW-skin-ITS-paired-rep-seqs.qza \
  --o-classification BNW-skin-unite-taxonomy-paired.qza

In [ ]:
qiime taxa barplot \
  --i-table BNW-skin-ITS-paired-reindexed-table.qza \
  --i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization taxa-bar-plots.qzv

In [ ]:
qiime metadata tabulate \
--m-input-file BNW-skin-unite-taxonomy-paired.qza \
--o-visualization BNW-skin-unite-taxonomy-paired.qzv

In [ ]:
# create rarefaction table 
qiime diversity alpha-rarefaction \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv   \
--o-visualization BNW-skin-ITS-paired-table-rarefaction.qzv \
--p-min-depth 500 \
--p-max-depth 50000

# plateaus around 12K reads

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 2198 \
--o-filtered-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza

# Remove toe samples from analysis (for the taxa barplot later on)

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "bodysite IN ('toe')" \
--p-exclude-ids \
--o-filtered-table BNW-skin-table-no-toe.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table BNW-skin-table-no-toe.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 2198 \
--o-filtered-table BNW-skin-table-no-toe-at-least-2198-reads.qza

In [ ]:
qiime feature-table summarize \
--i-table BNW-skin-table-no-toe-at-least-2198-reads.qza \
--o-visualization BNW-skin-table-no-toe-at-least-2198-reads.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table BNW-skin-table-no-toe-at-least-2198-reads.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization BNW-skin-table-no-toe-at-least-2198-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table BNW-skin-table-no-toe-at-least-2198-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-table-no-toe-SEVERITY.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-table-no-toe-SEVERITY.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization BNW-skin-table-no-toe-SEVERITY-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 5538 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table BNW-skin-table-no-toe-at-least-2198-reads.qza \
--i-phylogeny unite-rooted-tree.qza \
--p-sampling-depth 5538 \
--m-metadata-file BNW-skin-metadata.tsv \
--output-dir BNW-skin-Core-metrics-final-filtered-table-5538

In [ ]:
for i in BNW-skin-Core-metrics-final-filtered-table-5538/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata.tsv --o-visualization ${i/.qza/.qzv}

# Taxa bar plots

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization BNW-skin-paired-table-atleast-2198-reads-BARPLOT-PER-SAMPLE.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column infectionstatus \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-ITS-paired-table-atleast-2198-reads-INFECTION-STATUS.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads-INFECTION-STATUS.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-infection.tsv \
--o-visualization BNW-skin-ITS-paired-table-atleast-2198-reads-INFECTION-STATUS-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column bodysite \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-ITS-paired-table-atleast-2198-reads-BODYSITE.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads-BODYSITE.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-BS.tsv \
--o-visualization BNW-skin-ITS-paired-table-atleast-2198-reads-BODYSITE-BARPLOT.qzv

In [ ]:
qiime feature-table group \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table BNW-skin-ITS-paired-table-atleast-2198-reads-WOMBAT.qza

In [ ]:
qiime taxa barplot \
--i-table BNW-skin-ITS-paired-table-atleast-2198-reads-WOMBAT.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-WOM.tsv \
--o-visualization BNW-skin-ITS-paired-table-atleast-2198-reads-WOMBAT-BARPLOT.qzv

# Alpha/beta diversity

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences BNW-skin-ITS-paired-rep-seqs.qza \
  --o-alignment unite-aligned-rep-seqs.qza \
  --o-masked-alignment unite-masked-aligned-rep-seqs.qza \
  --o-tree unite-unrooted-tree.qza \
  --o-rooted-tree unite-rooted-tree.qza

In [ ]:
qiime diversity core-metrics-phylogenetic \
  --i-phylogeny unite-rooted-tree.qza \
  --i-table BNW-skin-ITS-paired-reindexed-table.qza \
  --p-sampling-depth 2198 \
  --m-metadata-file BNW-skin-metadata.tsv  \
  --output-dir core-metrics

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics/faith_pd_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization core-metrics/faith-pd-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics/evenness_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization core-metrics/evenness-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics/shannon_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization core-metrics/shannon-group-significance.qzv

In [ ]:
# alpha diversity
qiime diversity alpha-group-significance \
  --i-alpha-diversity core-metrics/observed_features_vector.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization core-metrics/observed-features-group-significance.qzv

In [ ]:
# beta-diversity between body sites
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/jaccard_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column bodysite \
  --o-visualization core-metrics/jaccard-distance-matrix-group-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/bray_curtis_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column bodysite \
  --o-visualization core-metrics/braycurtis-distance-matrix-group-significance.qzv \
  --p-pairwise

In [ ]:
# beta-diversity between wombats
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/jaccard_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization core-metrics/jaccard-distance-matrix-WOMBAT-group-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/bray_curtis_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column wombatid \
  --o-visualization core-metrics/braycurtis-distance-matrix-WOMBAT-group-significance.qzv \
  --p-pairwise

In [ ]:
# beta-diversity between infection status
qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/jaccard_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column infectionstatus \
  --o-visualization core-metrics/jaccard-distance-matrix-INFECTION-group-significance.qzv \
  --p-pairwise

qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics/bray_curtis_distance_matrix.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --m-metadata-column infectionstatus \
  --o-visualization core-metrics/braycurtis-distance-matrix-INFECTION-group-significance.qzv \
  --p-pairwise

# Remove toe samples and groups of Uncertain/low mange and Observational healthy from analysis

In [ ]:
qiime feature-table filter-samples \
--i-table BNW-skin-ITS-paired-reindexed-table.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[bodysite] IN ('toe') OR [Severity] IN ('Uncertain/low mange','Observational healthy')" \
--p-exclude-ids \
--o-filtered-table FinalCat/BNW-skin-table-no-toe_FinalCat.qza

In [ ]:
# remove samples with low reads
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-min-frequency 2198 \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qzv 

In [ ]:
# create taxa barplot per individual sample
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-at-least-2198-reads-BARPLOT-PER-SAMPLE.qzv
# w27 has a higher percentage of unidentified fungi when looking at level 2 (Genus), but I will keep it in.

In [ ]:
# collapse samples by severity and make barplots
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-SEVERITY_FinalCat.qza

In [ ]:
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-SEVERITY_FinalCat.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-SEVERITY.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVERITY-BARPLOT.qzv

In [ ]:
# collapse samples by wombat and make barplots
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-WOMBAT_FinalCat.qza

In [ ]:
qiime taxa barplot \
--i-table FinalCat/BNW-skin-table-no-toe-WOMBAT_FinalCat.qza \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--m-metadata-file BNW-skin-metadata-WOM.tsv \
--o-visualization FinalCat/BNW-skin-table-no-toe-WOMBAT-BARPLOT.qzv

In [ ]:
# core diversity on filtered table using 5538 rarefaction depth
qiime diversity core-metrics-phylogenetic \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--i-phylogeny unite-rooted-tree.qza \
--p-sampling-depth 5538 \
--m-metadata-file BNW-skin-metadata.tsv \
--output-dir FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538

In [ ]:
for i in FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/*vector.qza; do qiime diversity alpha-group-significance --i-alpha-diversity $i --m-metadata-file BNW-skin-metadata.tsv --o-visualization ${i/.qza/.qzv}; done

In [ ]:
qiime diversity alpha \
  --i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
  --p-metric chao1 \
  --o-alpha-diversity FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/chao1_index.qza

In [ ]:
qiime diversity alpha-group-significance \
  --i-alpha-diversity FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/chao1_index.qza \
  --m-metadata-file BNW-skin-metadata.tsv \
  --o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/chao1_index.qzv

# Permanova test for microbial composition differences

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/bray_curtis_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/bray_curtis_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

In [ ]:
qiime diversity beta-group-significance \
--i-distance-matrix FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/jaccard_distance_matrix.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column Severity \
--o-visualization FinalCat-BNW-skin-Core-metrics-final-filtered-table-5538/jaccard_PERMANOVA.qzv \
--p-method permanova \
--p-pairwise

# ANCOM analysis to identify differentially abundant features/OTUS

In [ ]:
#remove low abundance/prevalence features
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qzv

In [ ]:
qiime composition add-pseudocount \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--o-composition-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza

In [ ]:
qiime composition ancom \
--i-table FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--o-visualization FinalCat/BNW-skin-table-no-toe-FILTERED-ANCOM-ms8-mf500-pseudo-WOMBAT_FinalCat.qzv

# ANCOM collapsed at family level (i.e. not at the feature level)

In [ ]:
qiime taxa collapse \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza  \
--i-taxonomy BNW-skin-unite-taxonomy-paired.qza \
--p-level 5 \
--o-collapsed-table FinalCat/BNW-skin-table-no-toe-FAMILY_FinalCat.qza

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY_FinalCat.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500_FinalCat.qza \
--p-min-samples 8 \
--p-min-frequency 500

In [ ]:
qiime composition add-pseudocount \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500_FinalCat.qza  \
--o-composition-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza 

In [ ]:
qiime composition ancom \
--i-table FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--o-visualization FinalCat/BNW-skin-table-no-toe-FAMILY-FILTERED-ANCOM-ms8-mf500-pseudo-WOMBAT_FinalCat.qzv

# Unique feature analysis

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Confidently healthy'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Mangy'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-MANGYonly.qzv

In [ ]:
qiime feature-table filter-samples \
--i-table FinalCat/BNW-skin-table-no-toe-at-least-2198-reads_FinalCat.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--p-where "[Severity]='Severe mange'" \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVEREonly.qzv

# Provide overview of ASVs for each severity type

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--output-path FinalCat-table-no-toe-CONFHEALTHYonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-CONFHEALTHYonly.biom/feature-table.biom -o FinalCat-table-no-toe-CONFHEALTHYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--output-path FinalCat-table-no-toe-MANGYonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-MANGYonly.biom/feature-table.biom -o FinalCat-table-no-toe-MANGYonly.biom/feature-table.tsv --to-tsv

In [ ]:
qiime tools export \
--input-path FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--output-path FinalCat-table-no-toe-SEVEREonly.biom

In [ ]:
biom convert -i FinalCat-table-no-toe-SEVEREonly.biom/feature-table.biom -o FinalCat-table-no-toe-SEVEREonly.biom/feature-table.tsv --to-tsv

# Unique ASVs - min 3 samples due to severity groupings

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf191.qza \
--p-min-samples 3 \
--p-min-frequency 191

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf191.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf191.qzv

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf76.qza \
--p-min-samples 3 \
--p-min-frequency 76

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf76.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf76.qzv

In [ ]:
qiime feature-table filter-features \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly.qza \
--o-filtered-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf87.qza \
--p-min-samples 3 \
--p-min-frequency 87

In [ ]:
qiime feature-table summarize \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf87.qza \
--o-visualization FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf87.qzv

# Make taxa barplots

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHYonly-ms3-mf191.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf191-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-MANGYonly-ms3-mf76.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf76-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table group \
--i-table FinalCat/BNW-skin-table-no-toe-SEVEREonly-ms3-mf87.qza \
--m-metadata-file BNW-skin-metadata.tsv \
--m-metadata-column wombatid \
--p-mode mean-ceiling \
--p-axis sample \
--o-grouped-table FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf87-wombat-COLLAPSED.qza

In [ ]:
qiime feature-table merge \
--i-tables FinalCat/BNW-skin-table-no-toe-CONFHEALTHY-ms3-mf191-wombat-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-MANGY-ms3-mf76-wombat-COLLAPSED.qza FinalCat/BNW-skin-table-no-toe-SEVERE-ms3-mf87-wombat-COLLAPSED.qza \
--o-merged-table FinalCat/BNW-skin-table-no-toe-wombat-COLLASPED.qza